In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import glob

In [3]:
caffe_root = "/home/ubuntu/caffe/"
import sys
sys.path.insert(0, caffe_root + 'python')

In [4]:
import caffe

In [5]:
# Set the right path to your model definition file, pretrained model weights,
# and the image you would like to classify.
#MODEL_FILE = '../model/GoogLeNet/deploy.prototxt'
MODEL_FILE = 'model/GoogLeNet/deploy.prototxt'
PRETRAINED = 'model/GoogLeNet/plantvillage.caffemodel'
BINARY_PROTO_MEAN_FILE = "lmdb/mean.binaryproto"

"""
Replicated from https://github.com/BVLC/caffe/issues/290
"""
blob = caffe.proto.caffe_pb2.BlobProto()
data = open( BINARY_PROTO_MEAN_FILE  , 'rb' ).read()
blob.ParseFromString(data)
mean_arr = np.array( caffe.io.blobproto_to_array(blob) )[0]


##NOTE : If you do not have a GPU, you can uncomment the `set_mode_cpu()` call
#        instead of the `set_mode_gpu` call.
caffe.set_mode_gpu()
#caffe.set_mode_cpu()


net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       #mean=mean_arr, #.mean(1).mean(1),
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

In [7]:
input_images = []
y = []
for i in range(38):
    files = glob.glob("crowdai/c_%s/*.JPG"%i)
    for file_name in files[:1]:
        input_image = caffe.io.load_image(file_name)
        input_images.append(input_image)
        y.append(i)
    
prediction = net.predict(array(input_images))
print y
print list(prediction.argmax(axis=1))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]


In [8]:
for i, file_name in enumerate(glob.glob("./test/*.JPG")):
    input_image = caffe.io.load_image(file_name)[:256, :256, :]

In [80]:
file_paths = glob.glob("./test/*")
file_names = []
predictions = []

n = 0
step = 100
while n < len(file_paths):
    input_images = []
    for file_path in file_paths[n:n+step]:
        input_image = caffe.io.load_image(file_path)#[:256, :256, :]
        input_images.append(input_image)
        file_name = file_path.split("/")[-1]
        file_names.append(file_name)
    
    n += step
    print n
    pred = net.predict(array(input_images))
    predictions.extend(pred)
    assert len(file_names) == len(predictions)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [81]:
header == "filename," + ",".join(["c_%s"%i for i in range(38) ]) + "\n"

with open("predict.csv", "w") as f:
    f.write(header)
    for i, file_name in enumerate(file_names):
        pred = predictions[i]
        f.write(file_name+","+ ",".join(["{:.8f}".format(v).rstrip('0').rstrip('.') for v in pred]) + "\n")

In [82]:
!head -n 5 predict.csv

filename,c_0,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37
1b68cb1c4be48794fa1158868bbc15f7.jpg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.99999797,0,0,0,0.00000004,0,0.00000197,0,0,0
23a9c751e097c0bbd117f38c85a1fd2b.jpg,0.00014045,0,0.00000583,0,0,0,0,0,0,0.00000002,0,0,0,0,0,0,0.00020483,0,0,0,0,0,0,0,0,0.00000064,0,0,0.99961793,0.00000002,0.00000006,0,0.00002971,0,0,0.00000041,0,0
19a8494ca5d798d364134ef01c60c69d.jpg,0,0,0,0,0,0,0,0,0,0,0,0.45402312,0.54597682,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000001,0,0,0,0,0,0,0,0,0,0,0
d6ea99fc916ecb4d98093a8ed964ff2a.jpg,0,0,0,0.00004129,0.99995822,0.00000023,0,0,0,0,0,0,0,0,0,0.00000001,0,0,0,0,0,0.00000015,0,0,0.00000004,0,0,0,0,0,0,0,0,0,0,0,0,0


In [83]:
a = array(predictions)

In [95]:
b = array([a[i, v] for i, v in enumerate(a.argmax(axis=1))])

In [98]:
b.shape, b[b<0.9].shape, b[b<0.5].shape

((32388,), (2625,), (218,))